# Data Cleaning on NYPD Motor Vehicle Collision Data

First, I'll bring in the first three million rows of NYPD Motor Vehicle Collision Data using pandas. Then I'll take a look at the data itself, make a few diagnoses, and execute some data cleaning fixes to improve my dataset for analysis.

Let's begin by bringing in a few libraries we'll need:

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import sys
from IPython.core.display import display, HTML

In [3]:
datanyc = pd.read_csv("https://data.cityofnewyork.us/resource/h9gi-nx95.csv?$limit=3000000")

Firstly, I want to see the first few rows of my dataset. 

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
datanyc.head()

,date,time,borough,zip_code,latitude,longitude,location,on_street_name,off_street_name,cross_street_name,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,unique_key,vehicle_type_code1,vehicle_type_code2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5
0,2013-08-03T00:00:00.000,18:00,BROOKLYN,11223,40.591451,-73.976513,POINT (-73.9765134 40.5914514),86 STREET,WEST 6 STREET,NaN,0.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,NaN,NaN,NaN,115333,SPORT UTILITY / STATION WAGON,PASSENGER VEHICLE,NaN,NaN,NaN
1,2013-08-10T00:00:00.000,11:20,NaN,NaN,NaN,NaN,NaN,SEABURY STREET,54 AVENUE,NaN,0.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,NaN,NaN,NaN,257822,PASSENGER VEHICLE,PASSENGER VEHICLE,NaN,NaN,NaN
2,2013-07-19T00:00:00.000,17:30,BRONX,10453,40.861864,-73.912739,POINT (-73.9127392 40.8618642),WEST FORDHAM ROAD,MAJOR DEEGAN EXPRESSWAY,NaN,2.0,0.0,0,0,0,0,2,0,Unspecified,Unspecified,NaN,NaN,NaN,111939,PASSENGER VEHICLE,SPORT UTILITY / STATION WAGON,NaN,NaN,NaN
3,2013-07-19T00:00:00.000,17:10,QUEENS,11372,40.749745,-73.884309,POINT (-73.8843088 40.7497446),37 AVENUE,82 STREET,NaN,0.0,0.0,0,0,0,0,0,0,Unspecified,Unspecified,NaN,NaN,NaN,282673,PASSENGER VEHICLE,PASSENGER VEHICLE,NaN,NaN,NaN
4,2013-07-29T00:00:00.000,10:30,MANHATTAN,10022,40.761551,-73.966584,POINT (-73.9665842 40.7615508),EAST 59 STREET,3 AVENUE,NaN,0.0,0.0,0,0,0,0,0,0,Fatigued/Drowsy,Unspecified,NaN,NaN,NaN,35637,SPORT UTILITY / STATION WAGON,SPORT UTILITY / STATION WAGON,NaN,NaN,NaN


Let's get some overall information about the contents of the data.

In [5]:
pd.options.display.max_info_rows = 5000000
datanyc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1591797 entries, 0 to 1591796
Data columns (total 29 columns):
date                             1591797 non-null object
time                             1591797 non-null object
borough                          1109735 non-null object
zip_code                         1109544 non-null object
latitude                         1396631 non-null float64
longitude                        1396631 non-null float64
location                         1396631 non-null object
on_street_name                   1280491 non-null object
off_street_name                  1061438 non-null object
cross_street_name                219141 non-null object
number_of_persons_injured        1591780 non-null float64
number_of_persons_killed         1591766 non-null float64
number_of_pedestrians_injured    1591797 non-null int64
number_of_pedestrians_killed     1591797 non-null int64
number_of_cyclist_injured        1591797 non-null int64
number_of_cyclist_killed        

There are some missing values. Let's find the percentage of the missing values and see which columns have the most amount of missing values. I will get a mean of missing values and then round it to 4 decimal place.

In [6]:
pd.set_option('display.max_columns', 29)
datanyc.isnull().mean().round(4) * 100

date                              0.00
time                              0.00
borough                          30.28
zip_code                         30.30
latitude                         12.26
longitude                        12.26
location                         12.26
on_street_name                   19.56
off_street_name                  33.32
cross_street_name                86.23
number_of_persons_injured         0.00
number_of_persons_killed          0.00
number_of_pedestrians_injured     0.00
number_of_pedestrians_killed      0.00
number_of_cyclist_injured         0.00
number_of_cyclist_killed          0.00
number_of_motorist_injured        0.00
number_of_motorist_killed         0.00
contributing_factor_vehicle_1     0.26
contributing_factor_vehicle_2    13.41
contributing_factor_vehicle_3    93.53
contributing_factor_vehicle_4    98.66
contributing_factor_vehicle_5    99.66
unique_key                        0.00
vehicle_type_code1                0.33
vehicle_type_code2       

### What Needs Attention?

* I will not be using some columns (e.g. unique_key, on_street_name, off_street_name, cross_street_name) so I can drop them completely. 
* Some columns (such as vehicle_type_code_4, contributing_factor_vehicle_5) are nearly entirely empty. We'll definitely remove those as well. It is probably because there were no fourth or fifth vehicle contributing to the collision.
* 'Latitude' and 'longitude' columns seem to be contained in the 'location' column. I like keeping the two values separate for now, so we can probably remove 'location' later.
* The values that I expect to be 'datetime' type are object ('date' and 'time' columns). I'll fix those.
* I'm curious to see if collisions go up seasonally, so I'll make a new variable that bins the collisions by Spring (March, April, May), Summer (June, July, August), Fall (September, October, November), and Winter (December, January, February).
* I will rename some of the columns to make things easier while analyzing the data.
* I will change the the 'persons_injured' and 'persons_killed' column values from float to integer. 

Let's start! Just in case we mess anything up, we'll do all our cleaning in a new DataFrame called clean_nyc.

## Easy Fixes
We'll begin by removing some columns, keeping only those missing fewer than 30% of their values. We'll also change the zip_code to a string. We can also drop some columns we know we're not going to use. Those operations are simple enough that we'll do them all before checking in again on the DataFrame.

In [7]:
clean_nyc = datanyc.dropna(thresh=(0.30 * datanyc.shape[0]), axis=1).copy()

In [8]:
clean_nyc.loc[:,'zip_code'] = clean_nyc['zip_code'].astype(str)

In [9]:
clean_nyc.drop(columns=["unique_key", "on_street_name", "off_street_name"], inplace=True)

Let's take a peek at what clean_nyc looks like now, as far as data types and number of columns (and values in those columns):

In [10]:
clean_nyc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1591797 entries, 0 to 1591796
Data columns (total 19 columns):
date                             1591797 non-null object
time                             1591797 non-null object
borough                          1109735 non-null object
zip_code                         1591797 non-null object
latitude                         1396631 non-null float64
longitude                        1396631 non-null float64
location                         1396631 non-null object
number_of_persons_injured        1591780 non-null float64
number_of_persons_killed         1591766 non-null float64
number_of_pedestrians_injured    1591797 non-null int64
number_of_pedestrians_killed     1591797 non-null int64
number_of_cyclist_injured        1591797 non-null int64
number_of_cyclist_killed         1591797 non-null int64
number_of_motorist_injured       1591797 non-null int64
number_of_motorist_killed        1591797 non-null int64
contributing_factor_vehicle_1    1

And what about the percentage of the missing values now?

In [11]:
pd.set_option('display.max_columns', 29)
clean_nyc.isnull().mean().round(4) * 100

date                              0.00
time                              0.00
borough                          30.28
zip_code                          0.00
latitude                         12.26
longitude                        12.26
location                         12.26
number_of_persons_injured         0.00
number_of_persons_killed          0.00
number_of_pedestrians_injured     0.00
number_of_pedestrians_killed      0.00
number_of_cyclist_injured         0.00
number_of_cyclist_killed          0.00
number_of_motorist_injured        0.00
number_of_motorist_killed         0.00
contributing_factor_vehicle_1     0.26
contributing_factor_vehicle_2    13.41
vehicle_type_code1                0.33
vehicle_type_code2               16.44
dtype: float64

So far, so good.

## Lat/Long Data

The 'location' column is simply a concatenation of 'latitude' and 'longitude' columns.

In [12]:
clean_nyc[["latitude", "longitude", "location"]].head(30)

,latitude,longitude,location
0,40.591451,-73.976513,POINT (-73.9765134 40.5914514)
1,NaN,NaN,NaN
2,40.861864,-73.912739,POINT (-73.9127392 40.8618642)
3,40.749745,-73.884309,POINT (-73.8843088 40.7497446)
4,40.761551,-73.966584,POINT (-73.9665842 40.7615508)
5,40.744780,-73.975882,POINT (-73.9758819 40.74478)
6,40.643958,-73.937286,POINT (-73.937286 40.6439584)
7,40.799979,-73.942734,POINT (-73.9427338 40.7999787)
8,NaN,NaN,NaN
9,40.726071,-74.009907,POINT (-74.0099072 40.7260711)


Before we quickly do something, let's first see if it's true that all the location data follows the same pattern I see right now:

In [13]:
clean_nyc['location'].str.match('POINT \(-7\d\.\d+ \d{2}\.\d+\)', na=False).value_counts()

True     1395482
False     196315
Name: location, dtype: int64

196159 rows where that's not the case! More than I expected! Let's check them out!

In [14]:
clean_nyc[~clean_nyc['location'].str.match('POINT \(-7\d\.\d+ \d{2}\.\d+\)', na=False)]

,date,time,borough,zip_code,latitude,longitude,location,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,vehicle_type_code1,vehicle_type_code2
1,2013-08-10T00:00:00.000,11:20,NaN,nan,NaN,NaN,NaN,0.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,PASSENGER VEHICLE,PASSENGER VEHICLE
8,2013-08-10T00:00:00.000,20:00,NaN,nan,NaN,NaN,NaN,0.0,0.0,0,0,0,0,0,0,Failure to Yield Right-of-Way,Failure to Yield Right-of-Way,PASSENGER VEHICLE,PASSENGER VEHICLE
11,2013-07-23T00:00:00.000,11:30,NaN,nan,NaN,NaN,NaN,0.0,0.0,0,0,0,0,0,0,Unspecified,Unspecified,PASSENGER VEHICLE,PASSENGER VEHICLE
15,2013-08-08T00:00:00.000,18:01,NaN,nan,NaN,NaN,NaN,1.0,0.0,0,0,0,0,1,0,Unspecified,Unspecified,PASSENGER VEHICLE,PASSENGER VEHICLE
27,2013-07-27T00:00:00.000,20:00,NaN,nan,NaN,NaN,NaN,0.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,Driver Inattention/Distraction,PASSENGER VEHICLE,PASSENGER VEHICLE
47,2013-07-29T00:00:00.000,12:10,NaN,nan,NaN,NaN,NaN,0.0,0.0,0,0,0,0,0,0,Outside Car Distraction,Unspecified,OTHER,PASSENGER VEHICLE
49,2015-08-13T00:00:00.000,13:47,NaN,nan,NaN,NaN,NaN,0.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,SPORT UTILITY / STATION WAGON,VAN
52,2013-07-21T00:00:00.000,18:00,NaN,nan,NaN,NaN,NaN,1.0,0.0,0,0,0,0,1,0,Turning Improperly,Unspecified,TAXI,BUS
60,2013-08-14T00:00:00.000,17:10,NaN,nan,NaN,NaN,NaN,0.0,0.0,0,0,0,0,0,0,Unspecified,Unspecified,SPORT UTILITY / STATION WAGON,PICK-UP TRUCK
66,2013-08-11T00:00:00.000,13:40,NaN,nan,NaN,NaN,NaN,0.0,0.0,0,0,0,0,0,0,Unspecified,Unspecified,PASSENGER VEHICLE,PASSENGER VEHICLE


Ahh those sweet missing values... As we've seen before, latitude, longitude	and location columns have 12% of their values missing each. While we're at it, let's make some changes with the help of fillna function.

In [15]:
clean_nyc['location'] = clean_nyc['location'].fillna(0)
clean_nyc['latitude'] = clean_nyc['latitude'].fillna(0)
clean_nyc['longitude'] = clean_nyc['longitude'].fillna(0)

In [16]:
clean_nyc[~clean_nyc['location'].str.match('POINT \(-7\d\.\d+ \d{2}\.\d+\)', na=False)]

,date,time,borough,zip_code,latitude,longitude,location,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,vehicle_type_code1,vehicle_type_code2
1,2013-08-10T00:00:00.000,11:20,NaN,nan,0.0,0.0,0,0.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,PASSENGER VEHICLE,PASSENGER VEHICLE
8,2013-08-10T00:00:00.000,20:00,NaN,nan,0.0,0.0,0,0.0,0.0,0,0,0,0,0,0,Failure to Yield Right-of-Way,Failure to Yield Right-of-Way,PASSENGER VEHICLE,PASSENGER VEHICLE
11,2013-07-23T00:00:00.000,11:30,NaN,nan,0.0,0.0,0,0.0,0.0,0,0,0,0,0,0,Unspecified,Unspecified,PASSENGER VEHICLE,PASSENGER VEHICLE
15,2013-08-08T00:00:00.000,18:01,NaN,nan,0.0,0.0,0,1.0,0.0,0,0,0,0,1,0,Unspecified,Unspecified,PASSENGER VEHICLE,PASSENGER VEHICLE
27,2013-07-27T00:00:00.000,20:00,NaN,nan,0.0,0.0,0,0.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,Driver Inattention/Distraction,PASSENGER VEHICLE,PASSENGER VEHICLE
47,2013-07-29T00:00:00.000,12:10,NaN,nan,0.0,0.0,0,0.0,0.0,0,0,0,0,0,0,Outside Car Distraction,Unspecified,OTHER,PASSENGER VEHICLE
49,2015-08-13T00:00:00.000,13:47,NaN,nan,0.0,0.0,0,0.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,SPORT UTILITY / STATION WAGON,VAN
52,2013-07-21T00:00:00.000,18:00,NaN,nan,0.0,0.0,0,1.0,0.0,0,0,0,0,1,0,Turning Improperly,Unspecified,TAXI,BUS
60,2013-08-14T00:00:00.000,17:10,NaN,nan,0.0,0.0,0,0.0,0.0,0,0,0,0,0,0,Unspecified,Unspecified,SPORT UTILITY / STATION WAGON,PICK-UP TRUCK
66,2013-08-11T00:00:00.000,13:40,NaN,nan,0.0,0.0,0,0.0,0.0,0,0,0,0,0,0,Unspecified,Unspecified,PASSENGER VEHICLE,PASSENGER VEHICLE


Now all NaN's in the latitude and longitude columns are replaced with 0, let's change latitude, let's drop the location column.

In [17]:
clean_nyc.drop(columns="location", inplace = True)

And let's peek at our data..

In [18]:
clean_nyc.head()

,date,time,borough,zip_code,latitude,longitude,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,vehicle_type_code1,vehicle_type_code2
0,2013-08-03T00:00:00.000,18:00,BROOKLYN,11223.0,40.591451,-73.976513,0.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,SPORT UTILITY / STATION WAGON,PASSENGER VEHICLE
1,2013-08-10T00:00:00.000,11:20,NaN,nan,0.000000,0.000000,0.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,PASSENGER VEHICLE,PASSENGER VEHICLE
2,2013-07-19T00:00:00.000,17:30,BRONX,10453.0,40.861864,-73.912739,2.0,0.0,0,0,0,0,2,0,Unspecified,Unspecified,PASSENGER VEHICLE,SPORT UTILITY / STATION WAGON
3,2013-07-19T00:00:00.000,17:10,QUEENS,11372.0,40.749745,-73.884309,0.0,0.0,0,0,0,0,0,0,Unspecified,Unspecified,PASSENGER VEHICLE,PASSENGER VEHICLE
4,2013-07-29T00:00:00.000,10:30,MANHATTAN,10022.0,40.761551,-73.966584,0.0,0.0,0,0,0,0,0,0,Fatigued/Drowsy,Unspecified,SPORT UTILITY / STATION WAGON,SPORT UTILITY / STATION WAGON


In [19]:
clean_nyc['latitude'] = clean_nyc['latitude'].fillna(0)
clean_nyc['longitude'] = clean_nyc['longitude'].fillna(0)
clean_nyc.head()

,date,time,borough,zip_code,latitude,longitude,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,vehicle_type_code1,vehicle_type_code2
0,2013-08-03T00:00:00.000,18:00,BROOKLYN,11223.0,40.591451,-73.976513,0.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,SPORT UTILITY / STATION WAGON,PASSENGER VEHICLE
1,2013-08-10T00:00:00.000,11:20,NaN,nan,0.000000,0.000000,0.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,PASSENGER VEHICLE,PASSENGER VEHICLE
2,2013-07-19T00:00:00.000,17:30,BRONX,10453.0,40.861864,-73.912739,2.0,0.0,0,0,0,0,2,0,Unspecified,Unspecified,PASSENGER VEHICLE,SPORT UTILITY / STATION WAGON
3,2013-07-19T00:00:00.000,17:10,QUEENS,11372.0,40.749745,-73.884309,0.0,0.0,0,0,0,0,0,0,Unspecified,Unspecified,PASSENGER VEHICLE,PASSENGER VEHICLE
4,2013-07-29T00:00:00.000,10:30,MANHATTAN,10022.0,40.761551,-73.966584,0.0,0.0,0,0,0,0,0,0,Fatigued/Drowsy,Unspecified,SPORT UTILITY / STATION WAGON,SPORT UTILITY / STATION WAGON


## Date Format

Let's look at our dates to make sure they are all in the same format:

In [20]:
clean_nyc[['date', 'time']].head()

,date,time
0,2013-08-03T00:00:00.000,18:00
1,2013-08-10T00:00:00.000,11:20
2,2013-07-19T00:00:00.000,17:30
3,2013-07-19T00:00:00.000,17:10
4,2013-07-29T00:00:00.000,10:30


The 'date' column definitely needs some fixing...

I will transform the string timestamp for 'date' to a true datetime data type.

In [21]:
clean_nyc['date'] = pd.to_datetime(datanyc['date'])
clean_nyc.head()

,date,time,borough,zip_code,latitude,longitude,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,vehicle_type_code1,vehicle_type_code2
0,2013-08-03,18:00,BROOKLYN,11223.0,40.591451,-73.976513,0.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,SPORT UTILITY / STATION WAGON,PASSENGER VEHICLE
1,2013-08-10,11:20,NaN,nan,0.000000,0.000000,0.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,PASSENGER VEHICLE,PASSENGER VEHICLE
2,2013-07-19,17:30,BRONX,10453.0,40.861864,-73.912739,2.0,0.0,0,0,0,0,2,0,Unspecified,Unspecified,PASSENGER VEHICLE,SPORT UTILITY / STATION WAGON
3,2013-07-19,17:10,QUEENS,11372.0,40.749745,-73.884309,0.0,0.0,0,0,0,0,0,0,Unspecified,Unspecified,PASSENGER VEHICLE,PASSENGER VEHICLE
4,2013-07-29,10:30,MANHATTAN,10022.0,40.761551,-73.966584,0.0,0.0,0,0,0,0,0,0,Fatigued/Drowsy,Unspecified,SPORT UTILITY / STATION WAGON,SPORT UTILITY / STATION WAGON


I also want to create a new column which will carry the values for hours only. I think that can be helpful when grouping the times and visualizing the data. I will create a new column called 'hour' in which I will only have the hours instead of hours and minutes.

In [22]:
clean_nyc['time'] = pd.to_datetime(clean_nyc.time)
clean_nyc['hour'] = clean_nyc['time'].dt.hour
clean_nyc

,date,time,borough,zip_code,latitude,longitude,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,vehicle_type_code1,vehicle_type_code2,hour
0,2013-08-03,2019-10-28 18:00:00,BROOKLYN,11223.0,40.591451,-73.976513,0.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,SPORT UTILITY / STATION WAGON,PASSENGER VEHICLE,18
1,2013-08-10,2019-10-28 11:20:00,NaN,nan,0.000000,0.000000,0.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,PASSENGER VEHICLE,PASSENGER VEHICLE,11
2,2013-07-19,2019-10-28 17:30:00,BRONX,10453.0,40.861864,-73.912739,2.0,0.0,0,0,0,0,2,0,Unspecified,Unspecified,PASSENGER VEHICLE,SPORT UTILITY / STATION WAGON,17
3,2013-07-19,2019-10-28 17:10:00,QUEENS,11372.0,40.749745,-73.884309,0.0,0.0,0,0,0,0,0,0,Unspecified,Unspecified,PASSENGER VEHICLE,PASSENGER VEHICLE,17
4,2013-07-29,2019-10-28 10:30:00,MANHATTAN,10022.0,40.761551,-73.966584,0.0,0.0,0,0,0,0,0,0,Fatigued/Drowsy,Unspecified,SPORT UTILITY / STATION WAGON,SPORT UTILITY / STATION WAGON,10
5,2013-08-14,2019-10-28 08:14:00,MANHATTAN,10016.0,40.744780,-73.975882,0.0,0.0,0,0,0,0,0,0,Unspecified,Unspecified,PICK-UP TRUCK,PASSENGER VEHICLE,8
6,2013-07-22,2019-10-28 17:00:00,BROOKLYN,11203.0,40.643958,-73.937286,1.0,0.0,1,0,0,0,0,0,Unspecified,NaN,PASSENGER VEHICLE,NaN,17
7,2013-08-01,2019-10-28 16:20:00,MANHATTAN,10035.0,40.799979,-73.942734,1.0,0.0,0,0,0,0,1,0,Unspecified,Unspecified,SPORT UTILITY / STATION WAGON,SPORT UTILITY / STATION WAGON,16
8,2013-08-10,2019-10-28 20:00:00,NaN,nan,0.000000,0.000000,0.0,0.0,0,0,0,0,0,0,Failure to Yield Right-of-Way,Failure to Yield Right-of-Way,PASSENGER VEHICLE,PASSENGER VEHICLE,20
9,2013-07-26,2019-10-28 21:05:00,MANHATTAN,10014.0,40.726071,-74.009907,0.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,LIVERY VEHICLE,PASSENGER VEHICLE,21


Next, I will rename some of the columns to make things easier while analyzing the data.

In [23]:
clean_nyc.rename(columns={'number_of_persons_injured' : 'persons_injured',
                        'number_of_persons_killed' : 'persons_killed',
                        'number_of_pedestrians_injured' : 'pedestrians_injured',
                        'number_of_pedestrians_killed' : 'pedestrians_killed',
                        'number_of_cyclist_injured' : 'cyclist_injured',
                        'number_of_cyclist_killed' : 'cyclist_killed',
                        'number_of_motorist_injured'  : 'motorist_injured',
                        'number_of_motorist_killed' : 'motorist_killed'},inplace=True)
clean_nyc.head()

,date,time,borough,zip_code,latitude,longitude,persons_injured,persons_killed,pedestrians_injured,pedestrians_killed,cyclist_injured,cyclist_killed,motorist_injured,motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,vehicle_type_code1,vehicle_type_code2,hour
0,2013-08-03,2019-10-28 18:00:00,BROOKLYN,11223.0,40.591451,-73.976513,0.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,SPORT UTILITY / STATION WAGON,PASSENGER VEHICLE,18
1,2013-08-10,2019-10-28 11:20:00,NaN,nan,0.000000,0.000000,0.0,0.0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,PASSENGER VEHICLE,PASSENGER VEHICLE,11
2,2013-07-19,2019-10-28 17:30:00,BRONX,10453.0,40.861864,-73.912739,2.0,0.0,0,0,0,0,2,0,Unspecified,Unspecified,PASSENGER VEHICLE,SPORT UTILITY / STATION WAGON,17
3,2013-07-19,2019-10-28 17:10:00,QUEENS,11372.0,40.749745,-73.884309,0.0,0.0,0,0,0,0,0,0,Unspecified,Unspecified,PASSENGER VEHICLE,PASSENGER VEHICLE,17
4,2013-07-29,2019-10-28 10:30:00,MANHATTAN,10022.0,40.761551,-73.966584,0.0,0.0,0,0,0,0,0,0,Fatigued/Drowsy,Unspecified,SPORT UTILITY / STATION WAGON,SPORT UTILITY / STATION WAGON,10


Great! I can see that 'persons_injured' and 'persons_killed' column values are float. Let's change them from float to integer. To do so, I will first get rid of any missing values in the 'persons_injured' and 'persons_killed' columns.

In [24]:
clean_nyc.dropna(subset = ['persons_injured'], how='all', inplace=True)
clean_nyc.dropna(subset = ['persons_killed'], how='all', inplace=True)
clean_nyc['persons_injured'] = clean_nyc.persons_injured.astype(int)
clean_nyc['persons_killed'] = clean_nyc.persons_killed.astype(int)
clean_nyc.head()

,date,time,borough,zip_code,latitude,longitude,persons_injured,persons_killed,pedestrians_injured,pedestrians_killed,cyclist_injured,cyclist_killed,motorist_injured,motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,vehicle_type_code1,vehicle_type_code2,hour
0,2013-08-03,2019-10-28 18:00:00,BROOKLYN,11223.0,40.591451,-73.976513,0,0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,SPORT UTILITY / STATION WAGON,PASSENGER VEHICLE,18
1,2013-08-10,2019-10-28 11:20:00,NaN,nan,0.000000,0.000000,0,0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,PASSENGER VEHICLE,PASSENGER VEHICLE,11
2,2013-07-19,2019-10-28 17:30:00,BRONX,10453.0,40.861864,-73.912739,2,0,0,0,0,0,2,0,Unspecified,Unspecified,PASSENGER VEHICLE,SPORT UTILITY / STATION WAGON,17
3,2013-07-19,2019-10-28 17:10:00,QUEENS,11372.0,40.749745,-73.884309,0,0,0,0,0,0,0,0,Unspecified,Unspecified,PASSENGER VEHICLE,PASSENGER VEHICLE,17
4,2013-07-29,2019-10-28 10:30:00,MANHATTAN,10022.0,40.761551,-73.966584,0,0,0,0,0,0,0,0,Fatigued/Drowsy,Unspecified,SPORT UTILITY / STATION WAGON,SPORT UTILITY / STATION WAGON,10


Hmm I also want to make all my string values lower case.

In [38]:
clean_borough = clean_nyc['borough'].str.lower()

In [41]:
clean_vehicle_type_code1 = clean_nyc['vehicle_type_code1'].str.lower()

In [42]:
clean_vehicle_type_code2 = clean_nyc['vehicle_type_code2'].str.lower()

In [43]:
clean_nyc.loc[:, 'borough'] = clean_borough[0]
clean_nyc.loc[:, 'vehicle_type_code1'] = clean_vehicle_type_code1[0]
clean_nyc.loc[:, 'vehicle_type_code2'] = clean_vehicle_type_code2[0]
clean_nyc.head()

,date,time,borough,zip_code,latitude,longitude,persons_injured,persons_killed,pedestrians_injured,pedestrians_killed,cyclist_injured,cyclist_killed,motorist_injured,motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,vehicle_type_code1,vehicle_type_code2,hour
0,2013-08-03,2019-10-28 18:00:00,brooklyn,11223.0,40.591451,-73.976513,0,0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,sport utility / station wagon,passenger vehicle,18
1,2013-08-10,2019-10-28 11:20:00,brooklyn,nan,0.000000,0.000000,0,0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,sport utility / station wagon,passenger vehicle,11
2,2013-07-19,2019-10-28 17:30:00,brooklyn,10453.0,40.861864,-73.912739,2,0,0,0,0,0,2,0,Unspecified,Unspecified,sport utility / station wagon,passenger vehicle,17
3,2013-07-19,2019-10-28 17:10:00,brooklyn,11372.0,40.749745,-73.884309,0,0,0,0,0,0,0,0,Unspecified,Unspecified,sport utility / station wagon,passenger vehicle,17
4,2013-07-29,2019-10-28 10:30:00,brooklyn,10022.0,40.761551,-73.966584,0,0,0,0,0,0,0,0,Fatigued/Drowsy,Unspecified,sport utility / station wagon,passenger vehicle,10


I replaced the missing values in the 'latitude' and 'longitude' columns with zeros, now I also want to do something with the missing values in other columns..Let's see what we can do. Let's see what values we have in the 'contributing_factor_vehicle_1' and 'contributing_factor_vehicle_2' columns.

In [44]:
clean_nyc['contributing_factor_vehicle_1'].value_counts(dropna=False)

Unspecified                                              587009
Driver Inattention/Distraction                           295016
Failure to Yield Right-of-Way                             90253
Following Too Closely                                     78644
Backing Unsafely                                          60841
Other Vehicular                                           50639
Fatigued/Drowsy                                           46842
Turning Improperly                                        40768
Passing or Lane Usage Improper                            37114
Passing Too Closely                                       33999
Unsafe Lane Changing                                      30177
Traffic Control Disregarded                               24270
Driver Inexperience                                       23373
Lost Consciousness                                        19613
Pavement Slippery                                         15751
Prescription Medication                 

I think we can combine the missing and unspecified values.

In [45]:
clean_nyc['contributing_factor_vehicle_1'].fillna(value='Unspecified', inplace=True)

In [46]:
clean_nyc['contributing_factor_vehicle_1'].value_counts(dropna=False)

Unspecified                                              591170
Driver Inattention/Distraction                           295016
Failure to Yield Right-of-Way                             90253
Following Too Closely                                     78644
Backing Unsafely                                          60841
Other Vehicular                                           50639
Fatigued/Drowsy                                           46842
Turning Improperly                                        40768
Passing or Lane Usage Improper                            37114
Passing Too Closely                                       33999
Unsafe Lane Changing                                      30177
Traffic Control Disregarded                               24270
Driver Inexperience                                       23373
Lost Consciousness                                        19613
Pavement Slippery                                         15751
Prescription Medication                 

Let's see the values for the 'contributing_factor_vehicle_2' column.

In [47]:
clean_nyc['contributing_factor_vehicle_2'].value_counts(dropna=False)

Unspecified                                              1158595
NaN                                                       213460
Driver Inattention/Distraction                             72371
Other Vehicular                                            26568
Failure to Yield Right-of-Way                              13865
Following Too Closely                                      13333
Fatigued/Drowsy                                            10823
Passing or Lane Usage Improper                              9245
Turning Improperly                                          7683
Backing Unsafely                                            6966
Passing Too Closely                                         6490
Driver Inexperience                                         5426
Traffic Control Disregarded                                 5254
Lost Consciousness                                          5223
Unsafe Lane Changing                                        5204
Pavement Slippery        

We can do the same for this column as well.

In [48]:
clean_nyc['contributing_factor_vehicle_2'].fillna(value='Unspecified', inplace=True)
clean_nyc['contributing_factor_vehicle_1'].value_counts(dropna=False)

Unspecified                                              591170
Driver Inattention/Distraction                           295016
Failure to Yield Right-of-Way                             90253
Following Too Closely                                     78644
Backing Unsafely                                          60841
Other Vehicular                                           50639
Fatigued/Drowsy                                           46842
Turning Improperly                                        40768
Passing or Lane Usage Improper                            37114
Passing Too Closely                                       33999
Unsafe Lane Changing                                      30177
Traffic Control Disregarded                               24270
Driver Inexperience                                       23373
Lost Consciousness                                        19613
Pavement Slippery                                         15751
Prescription Medication                 

## Making a Seasons Variable

I'm interested in adding a variable that gives the season in which something occurred.

In [49]:
clean_nyc['date'] = pd.to_datetime(datanyc.date)
clean_nyc['date'].dt.month

0          8
1          8
2          7
3          7
4          7
5          8
6          7
7          8
8          8
9          7
10         8
11         7
12         8
13         7
14         7
15         8
16         8
17         7
18         7
19         7
20         8
21         8
22         8
23         7
24         8
25         7
26         8
27         7
28         8
29         7
30         7
31         8
32         7
33         8
34         7
35         8
36         8
37         8
38         8
39         8
40         7
41         7
42         8
43         8
44         7
45         7
46         7
47         7
48         7
49         8
50         7
51         8
52         7
53         7
54         8
55         8
56         7
57         8
58         7
59         8
60         8
61         8
62         8
63         8
64         8
65         7
66         8
67         7
68         7
69         7
70         8
71         7
72         8
73         7
74         7
75         7
76         8

In [50]:
def season(date):
    if date.month in ([3, 4, 5]):
        val = 'Spring'
    elif date.month in ([6, 7, 8]):
        val = 'Summer'
    elif date.month in ([9, 10, 11]):
        val = 'Autumn'
    elif date.month in ([12, 1, 2]):
        val = 'Winter'
    else:
        val = "Unspecified"
    return val

clean_nyc['season'] = clean_nyc['date'].apply(season)

In [51]:
clean_nyc['season'].value_counts()

Summer    431931
Autumn    417996
Spring    384120
Winter    357714
Name: season, dtype: int64

## Conclusion

We have a more precise set of data to work with in clean_nyc. Let's take a peek at it.

In [52]:
clean_nyc.shape

(1591761, 20)

In [53]:
datanyc.shape

(1591797, 29)

We've reduced our data by 10 columns and around 36 rows. Let's take a general look at our data as well as the 'info'.

In [54]:
clean_nyc.head()

,date,time,borough,zip_code,latitude,longitude,persons_injured,persons_killed,pedestrians_injured,pedestrians_killed,cyclist_injured,cyclist_killed,motorist_injured,motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,vehicle_type_code1,vehicle_type_code2,hour,season
0,2013-08-03,2019-10-28 18:00:00,brooklyn,11223.0,40.591451,-73.976513,0,0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,sport utility / station wagon,passenger vehicle,18,Summer
1,2013-08-10,2019-10-28 11:20:00,brooklyn,nan,0.000000,0.000000,0,0,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,sport utility / station wagon,passenger vehicle,11,Summer
2,2013-07-19,2019-10-28 17:30:00,brooklyn,10453.0,40.861864,-73.912739,2,0,0,0,0,0,2,0,Unspecified,Unspecified,sport utility / station wagon,passenger vehicle,17,Summer
3,2013-07-19,2019-10-28 17:10:00,brooklyn,11372.0,40.749745,-73.884309,0,0,0,0,0,0,0,0,Unspecified,Unspecified,sport utility / station wagon,passenger vehicle,17,Summer
4,2013-07-29,2019-10-28 10:30:00,brooklyn,10022.0,40.761551,-73.966584,0,0,0,0,0,0,0,0,Fatigued/Drowsy,Unspecified,sport utility / station wagon,passenger vehicle,10,Summer


In [55]:
clean_nyc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1591761 entries, 0 to 1591796
Data columns (total 20 columns):
date                             1591761 non-null datetime64[ns]
time                             1591761 non-null datetime64[ns]
borough                          1591761 non-null object
zip_code                         1591761 non-null object
latitude                         1591761 non-null float64
longitude                        1591761 non-null float64
persons_injured                  1591761 non-null int32
persons_killed                   1591761 non-null int32
pedestrians_injured              1591761 non-null int64
pedestrians_killed               1591761 non-null int64
cyclist_injured                  1591761 non-null int64
cyclist_killed                   1591761 non-null int64
motorist_injured                 1591761 non-null int64
motorist_killed                  1591761 non-null int64
contributing_factor_vehicle_1    1591761 non-null object
contributing_factor_ve

We've done a lot of data cleaning, and this is a great start for our next stage. We'll save our data locally.

In [ ]:
clean_nyc.to_csv("clean_nyc_data.csv", index = False)

### Thank you for reading!